# Performance Optimization

## Purpose

This notebook teaches you how to profile, optimize, and benchmark AM-QADF operations for better performance. You'll learn performance profiling, parallel execution strategies, memory optimization, caching, and benchmarking with interactive controls.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Profile operations for time and memory usage
- ✅ Optimize operations using parallel execution, caching, and lazy loading
- ✅ Benchmark operations and compare performance
- ✅ Apply memory optimization strategies
- ✅ Use Spark-based distributed processing
- ✅ Compare before/after performance improvements

## Estimated Duration

45-60 minutes

---

## Overview

Performance optimization enables efficient processing of large datasets:

- ⏱️ **Profiling**: Time and memory profiling to identify bottlenecks
- 🚀 **Optimization**: Parallel execution, caching, lazy loading, Spark processing
- 📊 **Benchmarking**: Performance comparison and scaling analysis
- 💾 **Memory Optimization**: Efficient memory usage strategies
- 🔄 **Caching**: Smart caching for repeated operations
- 📈 **Comparison**: Before/after performance analysis

Use the interactive widgets below to profile, optimize, and benchmark operations - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from typing import Optional, Tuple, Dict, Any, List

# Try to import profiling tools
try:
    import cProfile
    import pstats
    import io
    PROFILING_AVAILABLE = True
except ImportError:
    PROFILING_AVAILABLE = False
    print("⚠️ Profiling tools not available - using simplified profiling")

print("✅ Setup complete!")


✅ Setup complete!


## Interactive Performance Optimization Interface

Use the widgets below to profile, optimize, and benchmark operations. Select optimization mode, configure parameters, and visualize performance improvements interactively!


In [2]:
# Create Interactive Performance Optimization Interface

# Global state
profile_results = {}
optimization_results = {}
benchmark_results = {}
before_after_comparison = {}

# ============================================
# Top Panel: Optimization Mode and Actions
# ============================================

optimization_mode = RadioButtons(
    options=[
        ('Profile', 'profile'),
        ('Optimize', 'optimize'),
        ('Benchmark', 'benchmark'),
        ('Compare', 'compare')
    ],
    value='profile',
    description='Mode:',
    style={'description_width': 'initial'}
)

operation_selector = Dropdown(
    options=[
        ('Signal Mapping', 'signal_mapping'),
        ('Voxel Fusion', 'voxel_fusion'),
        ('Interpolation', 'interpolation'),
        ('Quality Assessment', 'quality'),
        ('Analytics', 'analytics')
    ],
    value='signal_mapping',
    description='Operation:',
    style={'description_width': 'initial'}
)

profile_button = Button(
    description='Execute Profiling',
    button_style='success',
    icon='search',
    layout=Layout(width='180px')
)

optimize_button = Button(
    description='Execute Optimization',
    button_style='primary',
    icon='bolt',
    layout=Layout(width='180px')
)

benchmark_button = Button(
    description='Run Benchmark',
    button_style='info',
    icon='chart-line',
    layout=Layout(width='160px')
)

top_panel = HBox([
    optimization_mode,
    operation_selector,
    profile_button,
    optimize_button,
    benchmark_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Optimization Configuration
# ============================================

# Profiling Configuration
profile_type = RadioButtons(
    options=[('Time', 'time'), ('Memory', 'memory'), ('Both', 'both')],
    value='both',
    description='Profile Type:',
    style={'description_width': 'initial'}
)
profile_depth = IntSlider(value=5, min=1, max=10, step=1, description='Depth:', style={'description_width': 'initial'})
profile_samples = IntSlider(value=100, min=1, max=1000, step=10, description='Samples:', style={'description_width': 'initial'})

profile_config = VBox([
    widgets.HTML("<b>Profiling Configuration:</b>"),
    profile_type,
    profile_depth,
    profile_samples
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Optimization Configuration
optimization_target = RadioButtons(
    options=[('Time', 'time'), ('Memory', 'memory'), ('Both', 'both')],
    value='both',
    description='Target:',
    style={'description_width': 'initial'}
)
optimization_parallel = Checkbox(value=True, description='Parallel Execution', style={'description_width': 'initial'})
optimization_caching = Checkbox(value=True, description='Caching', style={'description_width': 'initial'})
optimization_lazy = Checkbox(value=False, description='Lazy Loading', style={'description_width': 'initial'})
optimization_spark = Checkbox(value=False, description='Spark Processing', style={'description_width': 'initial'})
optimization_memory = Checkbox(value=True, description='Memory Optimization', style={'description_width': 'initial'})
optimization_workers = IntSlider(value=4, min=1, max=100, step=1, description='Workers:', style={'description_width': 'initial'})
optimization_cache_size = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Cache Size (GB):', style={'description_width': 'initial'})

optimization_config = VBox([
    widgets.HTML("<b>Optimization Configuration:</b>"),
    optimization_target,
    optimization_parallel,
    optimization_caching,
    optimization_lazy,
    optimization_spark,
    optimization_memory,
    optimization_workers,
    optimization_cache_size
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Benchmark Configuration
benchmark_operations = SelectMultiple(
    options=[
        ('Signal Mapping', 'signal_mapping'),
        ('Voxel Fusion', 'voxel_fusion'),
        ('Interpolation', 'interpolation'),
        ('Quality Assessment', 'quality')
    ],
    value=['signal_mapping', 'voxel_fusion'],
    description='Operations:',
    style={'description_width': 'initial'}
)
benchmark_data_sizes = SelectMultiple(
    options=[('Small (100K)', 'small'), ('Medium (1M)', 'medium'), ('Large (10M)', 'large')],
    value=['small', 'medium'],
    description='Data Sizes:',
    style={'description_width': 'initial'}
)
benchmark_iterations = IntSlider(value=10, min=1, max=100, step=1, description='Iterations:', style={'description_width': 'initial'})

benchmark_config = VBox([
    widgets.HTML("<b>Benchmark Configuration:</b>"),
    benchmark_operations,
    benchmark_data_sizes,
    benchmark_iterations
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Dynamic configuration accordion
config_accordion = Accordion(children=[
    profile_config,
    optimization_config,
    benchmark_config
])
config_accordion.set_title(0, 'Profiling')
config_accordion.set_title(1, 'Optimization')
config_accordion.set_title(2, 'Benchmark')

left_panel = VBox([
    widgets.HTML("<h3>Optimization Configuration</h3>"),
    config_accordion
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Optimization Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Profile', 'profile'), ('Optimization', 'optimization'), ('Benchmark', 'benchmark'), ('Comparison', 'comparison')],
    value='profile',
    description='View:',
    style={'description_width': 'initial'}
)

viz_output = Output(layout=Layout(height='600px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Optimization Visualization</h3>"),
    viz_mode,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Optimization Results
# ============================================

# Profile Results
profile_results_label = widgets.HTML("<b>Profile Results:</b>")
profile_results_display = widgets.HTML("No profiling performed yet")
profile_results_section = VBox([
    profile_results_label,
    profile_results_display
], layout=Layout(padding='5px'))

# Optimization Results
optimization_results_label = widgets.HTML("<b>Optimization Results:</b>")
optimization_results_display = widgets.HTML("No optimization performed")
optimization_results_section = VBox([
    optimization_results_label,
    optimization_results_display
], layout=Layout(padding='5px'))

# Benchmark Results
benchmark_results_label = widgets.HTML("<b>Benchmark Results:</b>")
benchmark_results_display = widgets.HTML("No benchmark performed")
benchmark_results_section = VBox([
    benchmark_results_label,
    benchmark_results_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_profile_button = Button(description='Export Profile', button_style='', layout=Layout(width='150px'))
export_config_button = Button(description='Export Config', button_style='', layout=Layout(width='150px'))
export_benchmark_button = Button(description='Export Benchmark', button_style='', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_profile_button,
    export_config_button,
    export_benchmark_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    profile_results_section,
    optimization_results_section,
    benchmark_results_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to optimize")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
info_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    info_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Optimization Functions
# ============================================

def execute_profiling(button):
    """Execute profiling operation."""
    global profile_results
    
    status_display.value = "<b>Status:</b> Profiling operation..."
    progress_bar.value = 0
    
    try:
        operation = operation_selector.value
        progress_bar.value = 20
        
        # Simulate profiling
        import time
        time.sleep(0.5)
        
        # Generate profile results
        profile_results = {
            'operation': operation,
            'total_time': 2.5,
            'time_breakdown': {
                'data_load': 0.5,
                'processing': 1.5,
                'output': 0.5
            },
            'memory_usage': {
                'peak': 512,  # MB
                'average': 256,
                'min': 128
            },
            'hot_spots': [
                ('interpolation', 1.2),
                ('voxel_indexing', 0.8),
                ('data_loading', 0.5)
            ],
            'profile_type': profile_type.value
        }
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Profiling completed</span>"
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during profiling</span>"
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        progress_bar.value = 0

def execute_optimization(button):
    """Execute optimization operation."""
    global optimization_results, before_after_comparison
    
    status_display.value = "<b>Status:</b> Optimizing operation..."
    progress_bar.value = 0
    
    try:
        operation = operation_selector.value
        progress_bar.value = 20
        
        # Simulate optimization
        import time
        time.sleep(0.5)
        
        # Calculate improvements based on optimization methods
        time_improvement = 0
        memory_improvement = 0
        
        if optimization_parallel.value:
            time_improvement += 0.3  # 30% improvement
        if optimization_caching.value:
            time_improvement += 0.2  # 20% improvement
        if optimization_lazy.value:
            memory_improvement += 0.25  # 25% improvement
        if optimization_memory.value:
            memory_improvement += 0.15  # 15% improvement
        
        # Generate optimization results
        optimization_results = {
            'operation': operation,
            'before': {
                'time': 2.5,
                'memory': 512
            },
            'after': {
                'time': 2.5 * (1 - time_improvement),
                'memory': 512 * (1 - memory_improvement)
            },
            'improvements': {
                'time_reduction': time_improvement * 100,
                'memory_reduction': memory_improvement * 100,
                'speedup': 1 / (1 - time_improvement) if time_improvement < 1 else 1
            },
            'methods_applied': [
                'parallel' if optimization_parallel.value else None,
                'caching' if optimization_caching.value else None,
                'lazy_loading' if optimization_lazy.value else None,
                'memory_opt' if optimization_memory.value else None
            ],
            'workers': optimization_workers.value if optimization_parallel.value else 1
        }
        
        before_after_comparison = optimization_results
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Optimization completed</span>"
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during optimization</span>"
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        progress_bar.value = 0

def execute_benchmark(button):
    """Execute benchmark operation."""
    global benchmark_results
    
    status_display.value = "<b>Status:</b> Running benchmark..."
    progress_bar.value = 0
    
    try:
        operations = list(benchmark_operations.value)
        data_sizes = list(benchmark_data_sizes.value)
        iterations = benchmark_iterations.value
        
        progress_bar.value = 10
        
        # Simulate benchmark
        import time
        benchmark_data = {}
        
        for op in operations:
            for size in data_sizes:
                times = []
                for i in range(iterations):
                    time.sleep(0.05)  # Simulate operation
                    times.append(np.random.uniform(1.0, 3.0))
                    progress_bar.value = 10 + int(80 * ((operations.index(op) * len(data_sizes) + data_sizes.index(size)) * iterations + i + 1) / (len(operations) * len(data_sizes) * iterations))
                
                key = f"{op}_{size}"
                benchmark_data[key] = {
                    'mean_time': np.mean(times),
                    'std_time': np.std(times),
                    'min_time': np.min(times),
                    'max_time': np.max(times),
                    'throughput': 1000 / np.mean(times)  # ops/sec
                }
        
        benchmark_results = benchmark_data
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Benchmark completed</span>"
        
        update_displays()
        update_visualization()
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during benchmark</span>"
        info_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        progress_bar.value = 0

def update_displays():
    """Update results displays."""
    global profile_results, optimization_results, benchmark_results
    
    # Profile results
    if profile_results:
        prof = profile_results
        prof_html = f"<p><b>Operation:</b> {prof['operation']}</p>"
        prof_html += f"<p><b>Total Time:</b> {prof['total_time']:.2f}s</p>"
        if 'time_breakdown' in prof:
            prof_html += "<p><b>Time Breakdown:</b></p><ul>"
            for key, value in prof['time_breakdown'].items():
                prof_html += f"<li>{key}: {value:.2f}s</li>"
            prof_html += "</ul>"
        if 'memory_usage' in prof:
            mem = prof['memory_usage']
            prof_html += f"<p><b>Memory:</b> Peak: {mem['peak']} MB</p>"
        profile_results_display.value = prof_html
    else:
        profile_results_display.value = "<p>No profiling performed yet</p>"
    
    # Optimization results
    if optimization_results:
        opt = optimization_results
        opt_html = f"<p><b>Operation:</b> {opt['operation']}</p>"
        if 'improvements' in opt:
            imp = opt['improvements']
            opt_html += f"<p><b>Time Reduction:</b> {imp['time_reduction']:.1f}%</p>"
            opt_html += f"<p><b>Memory Reduction:</b> {imp['memory_reduction']:.1f}%</p>"
            opt_html += f"<p><b>Speedup:</b> {imp['speedup']:.2f}x</p>"
        optimization_results_display.value = opt_html
    else:
        optimization_results_display.value = "<p>No optimization performed</p>"
    
    # Benchmark results
    if benchmark_results:
        bench_html = "<p><b>Benchmark Results:</b></p>"
        for key, value in list(benchmark_results.items())[:3]:
            bench_html += f"<p><b>{key}:</b></p>"
            bench_html += f"<p>  Mean: {value['mean_time']:.2f}s</p>"
            bench_html += f"<p>  Throughput: {value['throughput']:.1f} ops/s</p>"
        benchmark_results_display.value = bench_html
    else:
        benchmark_results_display.value = "<p>No benchmark performed</p>"

def update_visualization():
    """Update visualization display."""
    global profile_results, optimization_results, benchmark_results, before_after_comparison
    
    with viz_output:
        clear_output(wait=True)
        
        viz = viz_mode.value
        mode = optimization_mode.value
        
        if viz == 'profile' and profile_results:
            # Profile visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Time breakdown
            if 'time_breakdown' in profile_results:
                breakdown = profile_results['time_breakdown']
                axes[0].bar(breakdown.keys(), breakdown.values(), 
                           color=['blue', 'green', 'orange'], alpha=0.7)
                axes[0].set_ylabel('Time (s)')
                axes[0].set_title('Time Breakdown')
                axes[0].grid(True, alpha=0.3, axis='y')
            
            # Memory usage
            if 'memory_usage' in profile_results:
                mem = profile_results['memory_usage']
                axes[1].bar(['Peak', 'Average', 'Min'], 
                           [mem['peak'], mem['average'], mem['min']],
                           color=['red', 'orange', 'green'], alpha=0.7)
                axes[1].set_ylabel('Memory (MB)')
                axes[1].set_title('Memory Usage')
                axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'optimization' and optimization_results:
            # Optimization visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            opt = optimization_results
            if 'before' in opt and 'after' in opt:
                # Before/after comparison
                before_time = opt['before']['time']
                after_time = opt['after']['time']
                before_mem = opt['before']['memory']
                after_mem = opt['after']['memory']
                
                # Time comparison
                axes[0].bar(['Before', 'After'], [before_time, after_time],
                           color=['red', 'green'], alpha=0.7)
                axes[0].set_ylabel('Time (s)')
                axes[0].set_title('Time Optimization')
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Memory comparison
                axes[1].bar(['Before', 'After'], [before_mem, after_mem],
                           color=['red', 'green'], alpha=0.7)
                axes[1].set_ylabel('Memory (MB)')
                axes[1].set_title('Memory Optimization')
                axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'benchmark' and benchmark_results:
            # Benchmark visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Performance comparison
            operations = []
            mean_times = []
            throughputs = []
            
            for key, value in benchmark_results.items():
                operations.append(key.replace('_', ' ').title())
                mean_times.append(value['mean_time'])
                throughputs.append(value['throughput'])
            
            axes[0].bar(operations, mean_times, color='blue', alpha=0.7)
            axes[0].set_ylabel('Time (s)')
            axes[0].set_title('Benchmark: Execution Time')
            axes[0].tick_params(axis='x', rotation=45)
            axes[0].grid(True, alpha=0.3, axis='y')
            
            axes[1].bar(operations, throughputs, color='green', alpha=0.7)
            axes[1].set_ylabel('Throughput (ops/s)')
            axes[1].set_title('Benchmark: Throughput')
            axes[1].tick_params(axis='x', rotation=45)
            axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        
        elif viz == 'comparison' and before_after_comparison:
            # Comparison visualization
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            comp = before_after_comparison
            if 'before' in comp and 'after' in comp:
                # Performance improvement
                improvements = comp['improvements']
                axes[0].bar(['Time Reduction', 'Memory Reduction'], 
                           [improvements['time_reduction'], improvements['memory_reduction']],
                           color=['blue', 'green'], alpha=0.7)
                axes[0].set_ylabel('Improvement (%)')
                axes[0].set_title('Performance Improvements')
                axes[0].grid(True, alpha=0.3, axis='y')
                
                # Speedup
                axes[1].bar(['Speedup'], [improvements['speedup']],
                           color='orange', alpha=0.7)
                axes[1].set_ylabel('Speedup (x)')
                axes[1].set_title('Performance Speedup')
                axes[1].grid(True, alpha=0.3, axis='y')
                axes[1].axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Baseline')
                axes[1].legend()
            
            plt.tight_layout()
            plt.show()
        else:
            display(HTML("<p>Execute profiling/optimization/benchmark to see visualization</p>"))

# Update configuration visibility based on mode
def update_config_visibility(change):
    """Update which configuration section is visible."""
    mode = change['new']
    
    # Show relevant accordion section
    config_accordion.selected_index = {
        'profile': 0,
        'optimize': 1,
        'benchmark': 2,
        'compare': 1
    }.get(mode, 0)
    
    # Update viz mode
    if mode == 'profile':
        viz_mode.value = 'profile'
    elif mode == 'optimize':
        viz_mode.value = 'optimization'
    elif mode == 'benchmark':
        viz_mode.value = 'benchmark'
    elif mode == 'compare':
        viz_mode.value = 'comparison'

optimization_mode.observe(update_config_visibility, names='value')

# Connect events
profile_button.on_click(execute_profiling)
optimize_button.on_click(execute_optimization)
benchmark_button.on_click(execute_benchmark)
viz_mode.observe(lambda x: update_visualization(), names='value')
optimization_mode.observe(lambda x: update_visualization(), names='value')

# Auto-run benchmark when in benchmark mode
def auto_benchmark(change):
    """Auto-run benchmark when mode changes to benchmark."""
    if change['new'] == 'benchmark':
        # Don't auto-run, just update display
        pass

optimization_mode.observe(auto_benchmark, names='value')

# Initial updates
update_visualization()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to profile, optimize, and benchmark AM-QADF operations.

### Key Takeaways

1. **Profiling**: Time and memory profiling to identify bottlenecks and hot spots
2. **Optimization Methods**: Parallel execution, caching, lazy loading, Spark processing, memory optimization
3. **Optimization Targets**: Time optimization, memory optimization, or both
4. **Benchmarking**: Performance comparison across operations and data sizes
5. **Performance Metrics**: Execution time, throughput, memory usage, speedup
6. **Before/After Comparison**: Visualize performance improvements
7. **Optimization Configuration**: Configure workers, cache size, and optimization methods
8. **Benchmark Configuration**: Select operations, data sizes, and iterations

### Optimization Strategies

- **Parallel Execution**: Use multiple workers for parallel processing
- **Caching**: Cache frequently used data to avoid recomputation
- **Lazy Loading**: Load data only when needed
- **Spark Processing**: Use distributed processing for large datasets
- **Memory Optimization**: Optimize memory usage patterns

### Next Steps

Proceed to:
- **21_Custom_Extensions.ipynb** - Extending the framework with custom extensions
- **22_Troubleshooting_and_Debugging.ipynb** - Troubleshooting and debugging guide

### Related Resources

- Performance Documentation: `../docs/Tests/09-performance.md`
- Performance Tests: `../docs/Tests/04-test-categories/performance-tests.md`
- Examples: `../examples/`
